In [2]:
# Importa
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import os
from tqdm import tqdm
import sys
from folium.plugins import Search

In [3]:
# Carica il file CSV
file_path = './consultori-roma-mail-orari.csv'
data = pd.read_csv(file_path)

In [4]:
# Crea l'oggetto geocoder
geolocator = Nominatim(user_agent="consultori_mapper")

In [5]:
# Creazione della mappa di Roma
roma_map = folium.Map(location=[41.9028, 12.4964], zoom_start=12)  # Coordinate centrali di Roma

In [6]:
# Definisci colori per le diverse ASL
asl_colors = {
    'ROMA 1': 'blue',
    'ROMA 2': 'green',
    'ROMA 3': 'red',
    'ROMA 4': 'purple',
    'ROMA 5': 'orange',
    'ROMA 6': 'darkred'
}

icon_height = 37
icon_width = 35

In [7]:
# Crea feature group per tenere tutti i marker
marker_group = folium.FeatureGroup(name="Consultori")

In [8]:
# Aggiunta di marker per ogni consultorio
failed_entries = [] # Inizializza array per conservare le entries non caricate

for index, row in tqdm(data.iterrows(), total=len(data), desc="Aggiunta marker consultori", file=sys.stdout, ncols=100):
    print(data.columns)
    nome_consultorio = row['Consultorio Familiare']
    asl = row['ASL']
    raw_indirizzo = row['Indirizzo']
    cleaned_df = clean_indirizzo_column(df, column_name='Indirizzo')

    indirizzo = raw_indirizzo.lower().replace('snc', 's.n.c.').replace('c/o', '').replace('l.go', 'largo').replace("(C/O POL BARBERINI)","").strip()
    telefono = row['Telefono']
    orari = row['Orari']
    contact = row['Email/sito']

    # Ottieni le coordinate usando il geocoder
    try:
        location = geolocator.geocode(indirizzo + ", Italy")
        if location:
            lat, lon = location.latitude, location.longitude

            # Creazione del contenuto del popup
            if '@' in contact or contact == "n/d":
                popup_content = f"""
                          <h4>{nome_consultorio}</h4>
                          <i>{asl}</i>
                          <br>
                          <ul>
                              <li style='list-style-type: square'><b>Indirizzo</b>: {raw_indirizzo}
                              <li style='list-style-type: square'><b>Telefono</b>: {telefono}
                              <li style='list-style-type: square'><b>Email/sito</b>: {contact}
                          </ul>
                          <p>Orari apertura: <br>
                          {orari}
                          </p>
                            """
            else:
                popup_content = f"""
                          <h4>{nome_consultorio}</h4>
                          <i>{asl}</i>
                          <br>
                          <ul>
                              <li style='list-style-type: square'><b>Indirizzo</b>: {raw_indirizzo}
                              <li style='list-style-type: square'><b>Telefono</b>: {telefono}
                              <li style='list-style-type: square'><b>Email/sito</b>: <a href="https://{contact}" target="_blank">{contact}</a>
                          </ul>
                          <p>Orari apertura: <br>
                          {orari}
                          </p>
                            """

            # Construct the file path for the icon
            icon_image = f"./{asl_colors.get(asl)}.png"  # Example: "./red.svg"
            
            # Create a custom icon
            icon = folium.CustomIcon(
                icon_image,
                icon_size=(icon_width, icon_height),  # Adjust size as per your SVG dimensions
                icon_anchor=(22, 94),
                popup_anchor=(-3, -76),
            )
            
            # Add a marker with the custom icon
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_content, max_width=500),
                icon=icon  # Use the custom icon here
            ).add_to(marker_group)

            # Clear the line to avoid clutter
            sys.stdout.write("\033[F")
        # se un'entry non è stata geolocalizzata, aggiungila al'elenco dei fallimenti
        else: 
            print(f"Geocoding failed: {indirizzo}")
            failed_entries.append(row)
    except Exception as e:
        print(f"Error for {indirizzo}: {e}")
        failed_entries.append(row)

# Add a search bar linked to the feature group
Search(
    layer=marker_group,
    search_label="tooltip",  # Use marker tooltips (consultorio names) as searchable labels
    placeholder="Cerca consultorio...",
    collapsed=False  # Keep the search bar expanded
).add_to(roma_map)

# Add the feature group to the map
marker_group.add_to(roma_map)

Aggiunta marker consultori:   0%|                                            | 0/83 [00:00<?, ?it/s]Index(['ASL', 'Consultorio Familiare', 'Indirizzo', 'Telefono', 'Orari',
       'Email/sito'],
      dtype='object')
Geocoding failed: piazza s. m. della pieta` 5, 00135, roma
Aggiunta marker consultori:   1%|▍                                   | 1/83 [00:00<01:18,  1.05it/s]Index(['ASL', 'Consultorio Familiare', 'Indirizzo', 'Telefono', 'Orari',
       'Email/sito'],
      dtype='object')
Aggiunta marker consultori:   2%|▊                                   | 2/83 [00:01<01:15,  1.08it/s]Index(['ASL', 'Consultorio Familiare', 'Indirizzo', 'Telefono', 'Orari',
       'Email/sito'],
      dtype='object')
Aggiunta marker consultori:   4%|█▎                                  | 3/83 [00:02<01:04,  1.24it/s]Index(['ASL', 'Consultorio Familiare', 'Indirizzo', 'Telefono', 'Orari',
       'Email/sito'],
      dtype='object')
Aggiunta marker consultori:   5%|█▋                                  | 4/

In [9]:
# Log failed entries
if failed_entries:
    print("\nFailed geocoding entries:")
    for entry in failed_entries:
        print(entry)


Failed geocoding entries:
ASL                                                                 ROMA 1
Consultorio Familiare                 CONSULTORIO  FAMILIARE 'MONTE MARIO'
Indirizzo                         PIAZZA S. M. DELLA PIETA` 5, 00135, ROMA
Telefono                                                       06 60106696
Orari                    Lun-Ven 7:30-13:30\nLun, Mar e Gio 7:30-13:30/...
Email/sito               consultori14@aslroma1.it\nspaziogiovani14@aslr...
Name: 0, dtype: object
ASL                                                                 ROMA 2
Consultorio Familiare                         CONSULTORIO FAMILIARE RESEDE
Indirizzo                                 via Resede Acacie 1, 00171, ROMA
Telefono                                                  06 41436002-6020
Orari                             Lun-Ven 8:30-13:00 Mar e Mer 14:00-17:30
Email/sito               www.aslroma2.it/index.php/servizi-on-line-citt...
Name: 27, dtype: object
ASL                       

In [10]:
# Aggiungi una legenda alla mappa
legend_html = f'''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: fit-content; height: fit-content; 
            background-color: white; z-index:9999; font-size:14px; opacity: 0.8; border-radius: 6px;
            border:2px solid grey; padding: 10px;">
    <b>Legenda ASL</b><br>
    <img src="./blue.png" height={icon_height} width={icon_width}></img> ROMA 1<br>
    <img src="./green.png" height={icon_height} width={icon_width}></img> ROMA 2<br>
    <img src="./red.png" height={icon_height} width={icon_width}></img> ROMA 3<br>
    <img src="./purple.png" height={icon_height} width={icon_width}></img> ROMA 4<br>
    <img src="./orange.png" height={icon_height} width={icon_width}></img> ROMA 5<br>
    <img src="./darkred.png" height={icon_height} width={icon_width}></img> ROMA 6<br>
</div>
'''
roma_map.get_root().html.add_child(folium.Element(legend_html))

In [11]:
# Salvataggio della mappa in un file HTML nella stessa directory del file CSV
output_path = os.path.join(os.path.dirname(file_path), 'consultori_roma_map_searchbar.html')
roma_map.save(output_path)
print(f"Mappa salvata come {output_path}")

Mappa salvata come ./consultori_roma_map_searchbar.html
